In [ ]:
%%capture
!pip install bertopic

In [23]:
%%capture
!pip install --upgrade joblib==1.3.2 #1.1.0

In [10]:
!pip install pandas

In [12]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
import random
import json
import pandas as pd

In [ ]:
seed = 13
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [13]:
dat = pd.read_excel('/content/ReTweetsOnlyQuota.xlsx')

In [14]:
emp=[]
trp=[]
for i in range(len(dat)):
  if dat.iloc[i,13:22].to_numpy().sum()>0:
    trp.append(1)
  else:
    trp.append(0)

  if dat.iloc[i,22:].to_numpy().sum()>0:
    emp.append(1)
  else:
    emp.append(0)


In [15]:
dat['emplotment']=emp
dat['trope']=trp

In [16]:
dat_filtered = dat[['Tweet','emplotment','trope']]

In [17]:
dups=dat_filtered['Tweet'][dat_filtered['Tweet'].duplicated()]
dups.size

112

In [18]:
dat_filtered=dat_filtered.drop_duplicates(keep=False)

In [19]:
dat_filtered['target'] = list(dat_filtered[['emplotment','trope']].values.tolist())

In [20]:
text=dat_filtered['Tweet'].values.tolist()

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

In [24]:
from bertopic import BERTopic

In [49]:
#topic_model = BERTopic(language='english',calculate_probabilities=True, verbose=True,nr_topics=10)#nr_topics=6,,diversity=0.2,
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance

# Fine-tune your topic representations
representation_model = MaximalMarginalRelevance(diversity=0.5)
topic_model = BERTopic(representation_model=representation_model,calculate_probabilities=True)
topics, probs = topic_model.fit_transform(text)

2023-09-11 20:43:00,791 - BERTopic - Transformed documents to Embeddings
2023-09-11 20:43:06,972 - BERTopic - Reduced dimensionality
2023-09-11 20:43:07,160 - BERTopic - Clustered reduced embeddings


In [50]:
topic_model.update_topics(text, n_gram_range=(1, 2),vectorizer_model=vectorizer_model)

In [51]:
topic_model.reduce_topics(text, nr_topics=4)

2023-09-11 20:43:14,737 - BERTopic - Reduced number of topics from 26 to 4


In [52]:
freq = topic_model.get_topic_info(); freq.head(20)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,350,-1_whitegenocide_white_people_diversity,"[whitegenocide, white, people, diversity, coun...",[If Diversity is such a strength why is it for...
1,0,880,0_whitegenocide_white_diversity_antiwhite,"[whitegenocide, white, diversity, antiwhite, p...","[#whitegenocide https://t.co/7cSJeWwBK9, #Whit..."
2,1,42,1_kalergiplan_kalergiplan whitegenocide_thanks...,"[kalergiplan, kalergiplan whitegenocide, thank...",[#Maccabees #ThanksJews #ItsOkayToBeWhite #Kal...
3,2,19,2_racialrepublic_ipf_identitypoliticsforum ipf...,"[racialrepublic, ipf, identitypoliticsforum ip...",[#WhitePeople to be saved from #WhiteSupremacy...


In [53]:
topic_model.visualize_barchart(top_n_topics = 5, n_words = 10)

In [45]:
topic_model.visualize_hierarchy(top_n_topics=10)

In [46]:
topic_model.visualize_distribution(probs[20], min_probability=0.015)

In [47]:
topic_model.visualize_heatmap()